<a href="https://colab.research.google.com/github/PyBeginner1/MarketCapitalization/blob/main/MarketCapitalization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [180]:
import numpy as np 
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn import metrics

import warnings
warnings.filterwarnings(action='ignore')

In [41]:
data = pd.read_csv('/content/Fortune_1000.csv')

In [42]:
data.head()

,company,rank,rank_change,revenue,profit,num. of employees,sector,city,state,newcomer,ceo_founder,ceo_woman,profitable,prev_rank,CEO,Website,Ticker,Market Cap
0,Walmart,1,0.0,523964.0,14881.0,2200000,Retailing,Bentonville,AR,no,no,no,yes,1.0,C. Douglas McMillon,https://www.stock.walmart.com,WMT,411690
1,Amazon,2,3.0,280522.0,11588.0,798000,Retailing,Seattle,WA,no,yes,no,yes,5.0,Jeffrey P. Bezos,https://www.amazon.com,AMZN,1637405
2,Exxon Mobil,3,-1.0,264938.0,14340.0,74900,Energy,Irving,TX,no,no,no,yes,2.0,Darren W. Woods,https://www.exxonmobil.com,XOM,177923
3,Apple,4,-1.0,260174.0,55256.0,137000,Technology,Cupertino,CA,no,no,no,yes,3.0,Timothy D. Cook,https://www.apple.com,AAPL,2221176
4,CVS Health,5,3.0,256776.0,6634.0,290000,Health Care,Woonsocket,RI,no,no,yes,yes,8.0,Karen S. Lynch,https://www.cvshealth.com,CVS,98496


No. of rows & columns

In [43]:
data.shape

(1000, 18)

Check for null/missing values

In [44]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   company            1000 non-null   object 
 1   rank               1000 non-null   int64  
 2   rank_change        1000 non-null   float64
 3   revenue            1000 non-null   float64
 4   profit             998 non-null    float64
 5   num. of employees  1000 non-null   int64  
 6   sector             1000 non-null   object 
 7   city               1000 non-null   object 
 8   state              1000 non-null   object 
 9   newcomer           500 non-null    object 
 10  ceo_founder        1000 non-null   object 
 11  ceo_woman          1000 non-null   object 
 12  profitable         1000 non-null   object 
 13  prev_rank          1000 non-null   object 
 14  CEO                992 non-null    object 
 15  Website            1000 non-null   object 
 16  Ticker             938 no

In [45]:
data.isnull().sum()

company                0
rank                   0
rank_change            0
revenue                0
profit                 2
num. of employees      0
sector                 0
city                   0
state                  0
newcomer             500
ceo_founder            0
ceo_woman              0
profitable             0
prev_rank              0
CEO                    8
Website                0
Ticker                62
Market Cap            40
dtype: int64

In [148]:
print(X.shape, X_train.shape, X_test.shape)

(950, 459) (665, 458) (285, 458)


In [106]:
X.shape

(950, 10)

Check for missing values


In [107]:
X.isnull().sum()

revenue              0
profit               1
num. of employees    0
sector               0
city                 0
state                0
ceo_founder          0
ceo_woman            0
profitable           0
Market Cap           0
dtype: int64

Missing values in Target Column

In [108]:
X[X['Market Cap'].isnull()].index

Int64Index([], dtype='int64')

In [109]:
X.dtypes

revenue              float64
profit               float64
num. of employees      int64
sector                object
city                  object
state                 object
ceo_founder           object
ceo_woman             object
profitable            object
Market Cap           float64
dtype: object

Figuring out why market cap is object

In [82]:
X['Market Cap'].astype(np.float)

ValueError: ignored

We can see there are '-' in the target column

In [88]:
X['Market Cap'].replace('-', np.NaN).astype(np.float)

0       411690.0
1      1637405.0
2       177923.0
3      2221176.0
4        98496.0
         ...    
955        674.1
956        590.5
957      12059.3
958       1211.8
959        302.8
Name: Market Cap, Length: 960, dtype: float64

In [93]:
X['Market Cap'].replace('-', np.NaN).isnull().sum()

10

In [112]:
X.select_dtypes('object').value_counts()

sector               city        state  ceo_founder  ceo_woman  profitable
Financials           New York    NY     no           no         yes           21
Energy               Houston     TX     no           no         yes           15
Technology           San Jose    CA     no           no         yes           10
Energy               Houston     TX     no           no         no             9
Financials           Chicago     IL     no           no         yes            7
                                                                              ..
Materials            Fort Wayne  IN     yes          no         yes            1
                     Fort Mill   SC     no           no         yes            1
                     Evansville  IN     no           no         yes            1
                     Elkhart     IN     no           no         yes            1
Aerospace & Defense  Berwyn      PA     no           no         no             1
Length: 781, dtype: int64

In [116]:
{column : len(X[column].unique()) for column in X.select_dtypes('object').columns}

{'ceo_founder': 2,
 'ceo_woman': 2,
 'city': 386,
 'profitable': 2,
 'sector': 21,
 'state': 45}

In [118]:
X['ceo_founder'].nunique()

2

In [120]:
X['ceo_woman'].nunique()

2

In [121]:
X['ceo_woman'].value_counts()

no     889
yes     61
Name: ceo_woman, dtype: int64

In [125]:
pd.get_dummies(X['sector'])

,Aerospace & Defense,Apparel,Business Services,Chemicals,Energy,Engineering & Construction,Financials,Food & Drug Stores,"Food, Beverages & Tobacco",Health Care,"Hotels, Restaurants & Leisure",Household Products,Industrials,Materials,Media,Motor Vehicles & Parts,Retailing,Technology,Telecommunications,Transportation,Wholesalers
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
945,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
946,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
947,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
948,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0


Preprocessing

In [169]:
def preprocess_inputs(df):
  df = df.copy()

  #drop ununsed columns
  df = df.drop(['company','newcomer', 'prev_rank', 'CEO',	'Website', 	'Ticker', 'rank',	'rank_change'], axis = 1)

  #for missing values ('-') in target, encoding
  df['Market Cap'] = df['Market Cap'].replace('-', np.NaN).astype(np.float)

  #drop missing target rows
  missing_target_rows = df[df['Market Cap'].isnull()].index
  df = df.drop(missing_target_rows, axis = 0).reset_index(drop = True)

  #fill remaining missing values
  df['profit'] = df['profit'].fillna(df['profit'].mean())

  #encoding binary columns(yes/no)
  for column in ['ceo_founder',	'ceo_woman',	'profitable']:
    df[column] = df[column].replace({'no' : 0, 'yes' : 1})

  #one-hot encoding
  for column in ['sector',	'city',	'state']:
    dummies = pd.get_dummies(df[column], prefix = column)
    df = pd.concat([df, dummies], axis = 1)
    df = df.drop(column, axis = 1)
  
  #splitting data & target
  X = df.drop('Market Cap', axis = 1)
  Y = df['Market Cap']

  #train & test split
  X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, shuffle = True, random_state = 1)

  #scaling
  scaler = StandardScaler()
  scaler.fit(X_train)
  X_train = pd.DataFrame(scaler.transform(X_train), index = X_train.index,  columns = X_train.columns)
  X_test = pd.DataFrame(scaler.transform(X_test), index = X_test.index,  columns = X_test.columns)
  
  return  X_train, X_test, Y_train, Y_test

In [170]:
 X_train, X_test, Y_train, Y_test = preprocess_inputs(data)

In [171]:
X 

,revenue,profit,num. of employees,ceo_founder,ceo_woman,profitable,Market Cap,sector_Aerospace & Defense,sector_Apparel,sector_Business Services,sector_Chemicals,sector_Energy,sector_Engineering & Construction,sector_Financials,sector_Food & Drug Stores,"sector_Food, Beverages & Tobacco",sector_Health Care,"sector_Hotels, Restaurants & Leisure",sector_Household Products,sector_Industrials,sector_Materials,sector_Media,sector_Motor Vehicles & Parts,sector_Retailing,sector_Technology,sector_Telecommunications,sector_Transportation,sector_Wholesalers,city_Abbott Park,city_Akron,city_Allentown,city_Ames,city_Ankeny,city_Ann Arbor,city_Annapolis Junction,city_Anoka,city_Arlington,city_Armonk,city_Atlanta,city_Auburn Hills,...,state_CT,state_DC,state_DE,state_FL,state_GA,state_HI,state_IA,state_ID,state_IL,state_IN,state_KS,state_KY,state_LA,state_MA,state_MD,state_ME,state_MI,state_MN,state_MO,state_MS,state_NC,state_ND,state_NE,state_NH,state_NJ,state_NV,state_NY,state_OH,state_OK,state_OR,state_PA,state_PR,state_RI,state_SC,state_TN,state_TX,state_UT,state_VA,state_WA,state_WI
0,523964.0,14881.0,2200000,0,0,1,411690.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,280522.0,11588.0,798000,1,0,1,1637405.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,264938.0,14340.0,74900,0,0,1,177923.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,260174.0,55256.0,137000,0,0,1,2221176.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,256776.0,6634.0,290000,0,1,1,98496.0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
945,2007.0,274.0,9100,0,0,1,674.1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
946,1999.0,47.5,5100,0,0,1,590.5,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
947,1997.4,45.5,204,0,0,1,12059.3,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
948,1996.3,-95.1,5000,0,0,0,1211.8,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [172]:
print(X.shape, X_train.shape, X_test.shape)

(950, 459) (665, 458) (285, 458)


In [173]:
X_train.describe()

,revenue,profit,num. of employees,ceo_founder,ceo_woman,profitable,sector_Aerospace & Defense,sector_Apparel,sector_Business Services,sector_Chemicals,sector_Energy,sector_Engineering & Construction,sector_Financials,sector_Food & Drug Stores,"sector_Food, Beverages & Tobacco",sector_Health Care,"sector_Hotels, Restaurants & Leisure",sector_Household Products,sector_Industrials,sector_Materials,sector_Media,sector_Motor Vehicles & Parts,sector_Retailing,sector_Technology,sector_Telecommunications,sector_Transportation,sector_Wholesalers,city_Abbott Park,city_Akron,city_Allentown,city_Ames,city_Ankeny,city_Ann Arbor,city_Annapolis Junction,city_Anoka,city_Arlington,city_Armonk,city_Atlanta,city_Auburn Hills,city_Austin,...,state_CT,state_DC,state_DE,state_FL,state_GA,state_HI,state_IA,state_ID,state_IL,state_IN,state_KS,state_KY,state_LA,state_MA,state_MD,state_ME,state_MI,state_MN,state_MO,state_MS,state_NC,state_ND,state_NE,state_NH,state_NJ,state_NV,state_NY,state_OH,state_OK,state_OR,state_PA,state_PR,state_RI,state_SC,state_TN,state_TX,state_UT,state_VA,state_WA,state_WI
count,6.650000e+02,6.650000e+02,6.650000e+02,6.650000e+02,6.650000e+02,6.650000e+02,6.650000e+02,6.650000e+02,6.650000e+02,6.650000e+02,6.650000e+02,6.650000e+02,6.650000e+02,6.650000e+02,6.650000e+02,6.650000e+02,6.650000e+02,6.650000e+02,6.650000e+02,6.650000e+02,6.650000e+02,6.650000e+02,6.650000e+02,6.650000e+02,6.650000e+02,6.650000e+02,6.650000e+02,6.650000e+02,6.650000e+02,6.650000e+02,6.650000e+02,665.0,6.650000e+02,665.0,6.650000e+02,6.650000e+02,6.650000e+02,6.650000e+02,6.650000e+02,6.650000e+02,...,6.650000e+02,6.650000e+02,6.650000e+02,6.650000e+02,6.650000e+02,6.650000e+02,6.650000e+02,6.650000e+02,6.650000e+02,6.650000e+02,6.650000e+02,6.650000e+02,6.650000e+02,6.650000e+02,6.650000e+02,6.650000e+02,6.650000e+02,6.650000e+02,6.650000e+02,665.0,6.650000e+02,665.0,6.650000e+02,6.650000e+02,6.650000e+02,6.650000e+02,6.650000e+02,6.650000e+02,6.650000e+02,6.650000e+02,6.650000e+02,6.650000e+02,6.650000e+02,6.650000e+02,6.650000e+02,6.650000e+02,6.650000e+02,6.650000e+02,6.650000e+02,6.650000e+02
mean,-4.941745e-17,1.752984e-17,-5.342427e-18,-4.065253e-17,-1.251296e-16,3.706308e-17,1.335607e-17,-1.055129e-16,-1.085180e-17,1.182847e-16,-1.484193e-16,1.550138e-16,4.490977e-17,-9.057083e-18,-2.690413e-16,-2.520958e-17,-2.404092e-17,1.040938e-16,-5.843279e-19,3.664571e-17,7.362532e-17,7.679738e-18,-3.071895e-17,1.020070e-16,2.566869e-17,1.111058e-16,4.115338e-17,1.640292e-16,3.944005e-16,5.736431e-16,1.640292e-16,0.0,-2.708777e-16,0.0,1.640292e-16,-1.724185e-16,1.640292e-16,1.702898e-16,4.400824e-16,-7.043238e-17,...,9.182296e-18,3.759733e-16,2.462942e-16,1.238775e-16,-1.530104e-16,-2.761367e-16,2.050156e-16,-1.460820e-18,1.358980e-16,-1.215402e-16,-1.310147e-16,2.530975e-16,1.244201e-16,3.739699e-17,-2.742585e-16,1.640292e-16,1.969185e-16,1.502557e-17,6.077010e-17,0.0,-1.752149e-16,0.0,-1.941221e-16,-3.898302e-17,-1.142778e-16,-7.460615e-17,1.046782e-16,-2.779731e-17,2.408266e-16,3.866581e-16,4.532715e-17,-1.403013e-16,3.046853e-17,1.640292e-16,9.015345e-17,-3.157040e-16,1.640292e-16,-1.384857e-16,1.770096e-16,7.228971e-17
std,1.000753e+00,1.000753e+00,1.000753e+00,1.000753e+00,1.000753e+00,1.000753e+00,1.000753e+00,1.000753e+00,1.000753e+00,1.000753e+00,1.000753e+00,1.000753e+00,1.000753e+00,1.000753e+00,1.000753e+00,1.000753e+00,1.000753e+00,1.000753e+00,1.000753e+00,1.000753e+00,1.000753e+00,1.000753e+00,1.000753e+00,1.000753e+00,1.000753e+00,1.000753e+00,1.000753e+00,1.000753e+00,1.000753e+00,1.000753e+00,1.000753e+00,0.0,1.000753e+00,0.0,1.000753e+00,1.000753e+00,1.000753e+00,1.000753e+00,1.000753e+00,1.000753e+00,...,1.000753e+00,1.000753e+00,1.000753e+00,1.000753e+00,1.000753e+00,1.000753e+00,1.000753e+00,1.000753e+00,1.000753e+00,1.000753e+00,1.000753e+00,1.000753e+00,1.000753e+00,1.000753e+00,1.000753e+00,1.000753e+00,1.000753e+00,1.000753e+00,1.000753e+00,0.0,1.000753e+00,0.0,1.000753e+00,1.000753e+00,1.000753e+00,1.000753e+00,1.000753e+00

Training

Creating a dictionary of different models

In [174]:
models = {
    'Linear Regression'   : LinearRegression(),
    'Linear Regression L1': Lasso(),
    'Linear regression L2': Ridge(),
    'Decision Tree'       : DecisionTreeRegressor(),
    'Neural Network'      : MLPRegressor(),
    'Random Forest'       : RandomForestRegressor(),
    'Gradient Boosting'   : GradientBoostingRegressor()
}

for name, model in models.items():
  model.fit(X_train, Y_train)
  print(name+ ' trained!')

Linear Regression trained!
Linear Regression L1 trained!
Linear regression L2 trained!
Decision Tree trained!
Neural Network trained!
Random Forest trained!
Gradient Boosting trained!


**Results**

**Root Mean Square Error**

In [179]:
for name, model in models.items():
  y_predict = model.predict(X_test)
  rmse =np.sqrt(np.mean((Y_test-y_predict) ** 2))
  print(name+ 'RMSE {:.2f}'.format(rmse) )

Linear RegressionRMSE 10412989836944822272.00
Linear Regression L1RMSE 148364.67
Linear regression L2RMSE 150287.44
Decision TreeRMSE 122684.83
Neural NetworkRMSE 149366.65
Random ForestRMSE 91795.41
Gradient BoostingRMSE 91436.45


**R2 Squared Error**

In [184]:
for name, model in models.items():
  y_predict1 = model.predict(X_test)
  score = metrics.r2_score(Y_test, y_predict1)
  print(name + "R2 Score :{:.5f}".format(score))


Linear RegressionR2 Score :-5139649570991349529141313536.00000
Linear Regression L1R2 Score :-0.04338
Linear regression L2R2 Score :-0.07060
Decision TreeR2 Score :0.28655
Neural NetworkR2 Score :-0.05752
Random ForestR2 Score :0.60058
Gradient BoostingR2 Score :0.60370
